<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین سوم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش جست‌وجوی دارو</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
        <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>400200881</h5></td>
        <td style="border: none;"><h5>401204716</h5></td>
        <td style="border: none;"><h5>401204354</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> بهار ۱۴۰۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
    فایل ژوپیتر این تمرین در کولب توسعه داده و تست شده‌است. این فایل هم در محیط کولب، هم با ایمیج داکر jupyter/datascience-notebook تست شده‌است و همه قطعه‌کدها خروجی مورد انتظار را می‌دهند. اگه در بازتولید خروجی بعضی سل‌ها، یا کدهای تمرین مشکلی وجود داشت، ممنون می‌شویم در صورت امکان به ما اطلاع دهید تا فایل را در محیطی که قابل اجرا است، اجرا نموده و خروجی را نمایش دهیم. 
</b>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
در پوشه‌های دیتا و مدل، دیتاست تمرین، و مدل‌های ترین‌شده آن قرار دارند. با توجه به محدودیت حجم، برخی از این فایل‌ها وجود ندارد. در نتیجه، برای اجرای نوتبوک باید پروسه ترین را طی کرد که زمان‌بر است. لینک درایو زیر، مدل‌های ترین‌شده و دیتاست‌ها را در خود دارد و می‌توان آن‌ها را دانلود کرده و در پوشه‌های مربوطه کپی کرد. 

https://drive.google.com/drive/folders/1z-TFcZGJf9Y4iU3_TMvgvsp8C8DM_4Wo?usp=sharing
</b>


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### واردکردن و نصب کتاب‌خانه‌های مورد استفاده


In [96]:
try:
    import ipywidgets
except:
    %pip install ipywidgets 
        
try: 
    import pandas as pd
except:
    %pip install pandas 

try:
    import fuzzywuzzy
except:
    %pip install fuzzywuzzy
    
try: 
    import nltk
except: 
    %pip install nltk 
    
try:
    import gensim 
except:
    %pip install gensim 

try: 
    import langdetect
except: 
    %pip install langdetect 
    
try:
    from spellchecker import SpellChecker
except:
    %pip install pyspellchecker
    
try:
    import matplotlib as mpl
except:
    %pip install matplotlib 
    
try:
    import sklearn
except:
    %pip install sklearn
    
try:
    import scipy as sp
except:
    %pip install scipy
    
try:
    import numpy as np
except:
    
    %pip install numpy
    
try:
    import cupy
except ImportError:
    cupy = None
    

In [97]:
# Some configuration for pandas for a better display of tables
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

In [98]:
from IPython.display import HTML

# Set the font family and fallback fonts for pandas output globally

mpl.rcParams['font.family'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.sans-serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']
mpl.rcParams['font.serif'] = ['vazirmatn', 'Vazir', 'B Nazanin', 'Arial']

def set_pandas_font(fonts):
    css = f"""
    <style>
        table.dataframe td, table.dataframe th {{
            font-family: {fonts};
        }}
    </style>
    """
    return HTML(css)

set_pandas_font("'vazirmatn', 'Vazir', 'B Nazanin', 'Arial'")

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **خلاصه‌ای از صورت مسئله و راه‌حل**
اینجا یه خلاصه می‌دیم که مسئله چیه. بعد یه outline از کارایی که کردیم می‌دیم. 

کارایی که کردیم (حالا به شکل بهتری بیان میشن):‌
۱- یافتن دیتابیسی از داروهای انگلیسی
۲- نرمالیزه کردن دیتابیس به شکلی که می‌خوایم 
۳- یافتن معادل فارسی داروهای دیتابیس 


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **مراحل طراحی سامانه جستجوی دارو**
در این بخش، مراحلی که برای طراحی سامانه جستجوی دارو طی شده‌اند و در قسمت پیشین خلاصه‌ای از آن بیان شد، به تفصیل شرح داده خواهد شد. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۱. یافتن دیتابیس داروها به زبان انگلیسی

در وب‌سایت سازمان غذا و داروی ایران، دیتاستی با حدود ۵۰ هزار ورودی که برای هر دارو شامل نام دارو به فارسی و لاتین، نام جنریک آن، نام برند و موارد دیگر می‌باشد، برای دانلود در دسترس عام است. دیتابیس اصلی استفاده شده در این تمرین، این دیتابیس می‌باشد. البته بر روی این دیتابیس، تغییرات و نرمال‌سازی‌هایی انجام شده‌است که در ادامه ذکر خواهند شد. 
دیتابیس دیگر، از این مخزن گیت‌هاب دانلود شد. این دیتابیس، شامل نام هر دارو به ازای کد ATC آن می‌باشد. کد ATC دارو، که در دیتاست سازمان غذا و داروی ایران نیز موجود است، طبقه‌بندی داروها رو توصیف می‌کند. برای مثال، برای داروی ملاتونین که یک قرص تنظیم ساعت خواب است، کد ATC آن N05CH01 می‌باشد. در این کد، کاراکترها به ترتیب، سلسله‌مراتب دسته‌ای که دارو به آن متعلق است را مشخص می‌کنند. برای داروی ذکرشده، این سلسله مراتب به شرح زیر است:
- N: سیستم عصبی
- N05: داروهای روان‌پزشکی
- N05C: خواب‌آور و آرام‌بخش
- N05CH: آگونیست گیرنده‌های ملاتونین
- N05CH01: قرص ملاتونین ۲ میلی‌گرم

همان‌طور که از این مثال پیداست، کد ATC دارو اطلاعات ارزشمندی در خصوص دسته‌بندی، و کارکرد آن به ما می‌دهد. با توجه به اینکه دیتاست اصلی سازمان غذا و دارو، برای هر دارو کد ATC آن را نیز دارد، با Join نمودن این دو دیتاست، توانستیم برای هر دارو، دسته‌بندی و کاربردهایی از آن را در قالب چند ستون اضافه، به دیتاست اضافه نماییم. 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ۲. نرمال‌سازی و پیش‌پردازش‌های روی دیتابیس

### حذف داده‌های اضافه
دیتابیس دارهای سازمان غذا و دارو، دارای ستون‌هایی بود که برای این تمرین، کاربردی نداشتند. این ستون‌ها، شامل ستون‌هایی نظیر آمار فروش و قیمت داروها، و امکان فروش بدون نسخه داروها بودند. از آن‌جایی که این‌کار پیچیدگی خاصی نداشت، ستون‌های اضافه به صورت دستی از فایل حذف شدند. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### نرمال‌سازی داده‌ها
برخی نرمال‌سازی‌ها بر روی دیتاست داروها انجام گردید. مقدار تمام مدخل‌های جدول، به فرم lowercase آن تبدیل شد.  

In [99]:
IR_FDA_DATASET_PATH = "./data/Iran_FDA_1400_Dataset.csv"
WHO_ATC_INDEX_PATH = "./data/WHO_ATC_Index.csv"

medicine_ds = pd.read_csv(IR_FDA_DATASET_PATH)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,Actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR TABLET ORAL 200 mg,FAVIPIRAVIR,J05AX
1,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1mL 20 mg,"REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mg","REMDESIVIR INJECTION, SOLUTION PARENTERAL 5 mg/1mL 20 mL",REMDESIVIR,J05
2,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"NOLPAZA TABLET, DELAYED RELEASE ORAL 40 mg","PANTOPRAZOLE TABLET, DELAYED RELEASE ORAL 40 mg",PANTOPRAZOLE,A02BC02
3,Actoverco,اکتوورکو,"Krka, D. D., Novo Mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,ASENTRA TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE) TABLET ORAL 50 mg,SERTRALINE (AS HYDROCHLORIDE),N06AB06
4,Actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1mL 0.4 mL,NOXPRIN INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM INJECTION PARENTERAL 100 mg/1mL 0.4 mL,ENOXAPARIN SODIUM,B01AB05


In [100]:
atc_index_ds = pd.read_csv(WHO_ATC_INDEX_PATH)

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,A,ALIMENTARY TRACT AND METABOLISM,NaN,NaN,NaN,NaN
1,A01,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
2,A01A,STOMATOLOGICAL PREPARATIONS,NaN,NaN,NaN,NaN
3,A01AA,Caries prophylactic agents,NaN,NaN,NaN,NaN
4,A01AA01,sodium fluoride,1.1,mg,O,0.5 mg fluoride


In [101]:
import pandas as pd
from langdetect import detect

# Lowercase all entries in the drug dataset
medicine_ds[medicine_ds.select_dtypes(['object']).columns] = medicine_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b01ab05


In [102]:
# Lowercase all entries in the ATC index dataset
atc_index_ds[atc_index_ds.select_dtypes(['object']).columns] = atc_index_ds.select_dtypes(['object']).apply(lambda x: x.str.lower())

atc_index_ds.head()

,atc_code,atc_name,ddd,uom,adm_r,note
0,a,alimentary tract and metabolism,NaN,NaN,NaN,NaN
1,a01,stomatological preparations,NaN,NaN,NaN,NaN
2,a01a,stomatological preparations,NaN,NaN,NaN,NaN
3,a01aa,caries prophylactic agents,NaN,NaN,NaN,NaN
4,a01aa01,sodium fluoride,1.1,mg,o,0.5 mg fluoride


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### جداسازی تکه‌های معنادار کد ATC داروها
همان‌طور که در بالا گفته شد، کد ATC داروها شامل قسمت‌های معناداری است که به صورت سلسله‌مراتبی، طبقه‌بندی دارو،‌و حوزه استفاده آن را مشخص می‌کند. در این قسمت، کد ATC هر دارو، مطابق مستنداتی که ساختار کد ATC را شرح داده‌اند، به ۵ قسمت تقسیم شده و این ۵ قسمت، در قالب ۵ ستون (ویژگی) به دیتاست اصلی اضافه شدند. در زیر، کد مربوط به تقسیم شناسه ATC داروها پیاده‌سازی شده‌است. 


In [103]:
def extract_atc_parts(atc_code):
    atc_code = str(atc_code)
    return [atc_code[:1], atc_code[:3] if len(atc_code) >= 3 else None,
            atc_code[:4] if len(atc_code) >= 4 else None, atc_code[:5] if len(atc_code) >= 5 else None,
            atc_code if len(atc_code) >= 7 else None]
 
ATC_CATEGORIES = ['anatomical', 'therapeutic', 'pharmacological', 'subpharmacological', 'chemical']
medicine_atc_splitted = medicine_ds['atc_code'].apply(extract_atc_parts).to_list()

medicine_atc_columns = pd.DataFrame(medicine_atc_splitted, dtype='object', columns=ATC_CATEGORIES)

medicine_ds = medicine_ds.join(medicine_atc_columns, how='inner')
medicine_ds.rename(columns={'atc_code': 'atc_code_raw'}, inplace=True)

medicine_ds.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical,therapeutic,pharmacological,subpharmacological,chemical
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax,j,j05,j05a,j05ax,None
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05,j,j05,None,None,None
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,a,a02,a02b,a02bc,a02bc02
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06,n,n06,n06a,n06ab,n06ab06
4,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,ناکسپرین تزریقی پرنترال 100 mg/1ml 0.4 ml,noxprin injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium injection parenteral 100 mg/1ml 0.4 ml,enoxaparin sodium,b01ab05,b,b01,b01a,b01ab,b01ab05


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۳. ترکیب دیتاست داروها با دیتاست طبقه‌بندی کد ATC داروها
اینجا توضیح میدیم دو تا دیتاست چجوری جوین شدن

In [104]:
import pandas as pd

atc_columns = ['anatomical', 'therapeutic', 'pharmacological', 'subpharmacological', 'chemical']
desc_columns = ['anatomical_desc', 'therapeutic_desc', 'pharmacological_desc', 'subpharmacological_desc', 'chemical_desc']

medicine_ds_merged = medicine_ds.copy()

for atc_col, desc_col in zip(atc_columns, desc_columns):
    medicine_ds_merged = medicine_ds_merged.merge(atc_index_ds[['atc_code', 'atc_name']], left_on=atc_col, right_on='atc_code', how='left')
    medicine_ds_merged = medicine_ds_merged.rename(columns={'atc_name': desc_col})
    medicine_ds_merged = medicine_ds_merged.drop(columns=['atc_code'])

medicine_ds_merged = medicine_ds_merged.drop(columns=atc_columns)
medicine_ds_merged.head()

,pharma_company,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical_desc,therapeutic_desc,pharmacological_desc,subpharmacological_desc,chemical_desc
0,actero middleeast,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax,antiinfectives for systemic use,antivirals for systemic use,direct acting antivirals,other antivirals,NaN
1,actoverco,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05,antiinfectives for systemic use,antivirals for systemic use,NaN,NaN,NaN
2,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
3,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
4,actoverco,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06,nervous system,psychoanaleptics,antidepressants,selective serotonin reuptake inhibitors,sertraline


In [105]:
medicine_ds_merged.to_csv("./data/medicine_ds_merged.csv", index=False)

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۴. جداسازی دیتابیس‌های فارسی و انگلیسی
تا این مرحله، دیتاستی که داریم،‌شامل ستون‌هایی است که یا به زبان انگلیسی نوشته‌شده‌اند، یا به زبان فارسی. برای بیشتر ستون‌هایی که داده‌های آن‌ها در یک زبان است، ستون دیگری که معادل داده‌ها در زبان دیگر را داشته‌باشد وجود ندارد. در بخش‌های بعد توضیح داده‌خواهد شد که چگونه با تکنیک‌های Transfer Learning، دیتای فارسی که ناکامل بود، با استفاده از مدل ترین‌شده روی دیتای انگلیسی، ترین شد تا قدرتی معادل مدل انگلیسی داشته‌باشد. 

اما در این بخش، به عنوان نخستین گام، تکه‌های فارسی انگلیسی دیتاست از یکدیگر جدا می‌شوند. به این معنا که ستون‌هایی که فارسی هستند در یک دیتافریم، و ستون‌های انگلیسی در دیتافریم دیگر ذخیره شده، و هر یک از آن‌ها در یک فایل csv جداگانه ذخیره می‌گردند. 

In [107]:
persian_columns = ['supplier', 'brand_owner', 'distributor', 'manufacturer_country', 'brand_name_fa']
english_columns = medicine_ds_merged.columns.difference(persian_columns, sort=False)

# Select the Persian columns for persian_df
persian_df = medicine_ds_merged[persian_columns]

# Select the remaining columns for english_df
english_df = medicine_ds_merged[english_columns]


In [108]:
persian_df.head()

,supplier,brand_owner,distributor,manufacturer_country,brand_name_fa
0,اکتوورکو,اکتوورکو,الیت دارو,ایران,فاویپیراویر قرص خوراکی 200 mg
1,اکتوورکو,اکتوورکو,الیت دارو,ایران,رمدسیویر محلول تزریقی پرنترال 5 mg/1ml 20 mg
2,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg
3,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,نولپازا قرص انتریک کوتد خوراکی 40 mg
4,اکتوورکو,"krka, d. d., novo mesto",الیت دارو,ایران,آسنترا قرص خوراکی 50 mg


In [109]:
english_df.head()


,pharma_company,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical_desc,therapeutic_desc,pharmacological_desc,subpharmacological_desc,chemical_desc
0,actero middleeast,favipiravir tablet oral 200 mg,favipiravir tablet oral 200 mg,favipiravir,j05ax,antiinfectives for systemic use,antivirals for systemic use,direct acting antivirals,other antivirals,NaN
1,actoverco,"remdesivir injection, solution parenteral 5 mg/1ml 20 mg","remdesivir injection, solution parenteral 5 mg/1ml 20 ml",remdesivir,j05,antiinfectives for systemic use,antivirals for systemic use,NaN,NaN,NaN
2,actoverco,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
3,actoverco,"nolpaza tablet, delayed release oral 40 mg","pantoprazole tablet, delayed release oral 40 mg",pantoprazole,a02bc02,alimentary tract and metabolism,drugs for acid related disorders,drugs for peptic ulcer and gastro-oesophageal reflux disease (gord),proton pump inhibitors,pantoprazole
4,actoverco,asentra tablet oral 50 mg,sertraline (as hydrochloride) tablet oral 50 mg,sertraline (as hydrochloride),n06ab06,nervous system,psychoanaleptics,antidepressants,selective serotonin reuptake inhibitors,sertraline


In [110]:
persian_df.to_csv('./data/persian_df.csv', index=False)
english_df.to_csv('./data/english_df.csv', index=False)

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۵. آموزش مدل‌های زبانی
دیتاست هر یک از زبان‌ها، با مدل Word2Vec از کتاب‌خانه gensim، ترین شده و امبدینگ‌های آن در فایلی ذخیره شد. (برای اینکه مدل skipgram باشد، آرگومان sg را ست می‌کنیم. )

In [111]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Javad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Javad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Javad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Javad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [112]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
spell = SpellChecker()

def preprocess_row(row):
    row = row.str.lower()
    row = row.fillna(' ').astype(str)
    # Tokenize the text
    tokens = word_tokenize(' '.join(row))    
    # Remove stop words
    filtered_tokens = [w for w in tokens if not w in stop_words]
    # stemmed_tokens = [stemmer.stem(w) if w is not None else None for w in filtered_tokens]
    result_tokens = filtered_tokens
    return result_tokens

def preprocess(data):
    sentences = np.array(data.apply(preprocess_row, axis=1))
    return sentences


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
### ۵.۱. آموزش مدل روی دیتاست انگلیسی

In [113]:
import pandas as pd
from gensim.models import Word2Vec

# Load the dataset
data_en = pd.read_csv('./data/english_df.csv')
# data_en = data_en.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

sentences_en = preprocess(data_en)

In [114]:
import os

data_en['preprocessed_index'] = np.arange(sentences_en.shape[0])
data_en.to_csv('./data/english_df.csv', index=False)

if os.path.exists("./model/en_skipgram.model") and os.path.exists("./model/en_embeddings.txt"):
    print("Models for training English skipgram already exist. Skipping the training.")
else:
    # Train the skip-gram model
    model_en = Word2Vec(sentences_en, min_count=1, sg=1)

    # Save the model
    model_en.save('./model/en_skipgram.model')
    model_en.wv.save_word2vec_format('./model/en_embeddings.txt', binary=False)



Models for training English skipgram already exist. Skipping the training.


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
### ۵.۱. آموزش مدل روی دیتاست فارسی

In [115]:
import os

data_fa = pd.read_csv('./data/persian_df.csv')
sentences_fa = preprocess(data_fa)

data_fa['preprocessed_index'] = np.arange(sentences_fa.shape[0])
data_fa.to_csv('./data/persian_df.csv', index=False)

if os.path.exists("./model/fa_skipgram.model") and os.path.exists("./model/fa_embeddings.txt"):
    print("Models for training Farsi skipgram already exist. Skipping the training.")
else:
    # Train the skip-gram model
    model_fa = Word2Vec(sentences_fa, min_count=1, sg=1)

    # Save the model
    model_fa.save('./model/fa_skipgram.model')
    model_fa.wv.save_word2vec_format('./model/fa_embeddings.txt', binary=False)

Models for training Farsi skipgram already exist. Skipping the training.


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۶. مشترک‌سازی فضای معنی بین امبدینگ‌های دو زبان 
برای مشترک‌سازی فضای معنی بین دو زبان، از ابزار vecmap استفاده می‌کنیم. این ابزار به صورت کتابخانه Pypl در دسترس نیست  و در نتیجه باید مستقیما از مخزن گیت‌هاب کلون شود. پس از کلون،‌ به صورت unsupervised زبان‌ها را به هم مپ می‌کنیم. این مپینگ، به صورت خطی بوده و نتیجه آن نیز در فایل تکستی ذخیره می‌گردد که بعدا از آن استفاده می‌کنیم. 

In [116]:
!git clone https://github.com/artetxem/vecmap.git

fatal: destination path 'vecmap' already exists and is not an empty directory.


In [117]:
import os

if os.path.exists("./model/en_mapped_embeddings.txt") and os.path.exists("./model/fa_mapped_embeddings.txt"):
    print("Models for mapping embeddings already exist. Skipping the training.")
else:
    if cupy is not None:
        %%python ./vecmap/map_embeddings.py --unsupervised --cuda ./model/en_embeddings.txt ./model/fa_embeddings.txt ./model/en_mapped_embeddings.txt ./model/fa_mapped_embeddings.txt

    else:
        %%python ./vecmap/map_embeddings.py --unsupervised ./model/en_embeddings.txt ./model/fa_embeddings.txt ./model/en_mapped_embeddings.txt ./model/fa_mapped_embeddings.txt



Models for mapping embeddings already exist. Skipping the training.


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۷. جست‌و‌جوی اطلاعات دارویی با ورودی دو زبانه 
در ابتدا امبدینگ‌های مپ‌شده به زبان مشترک را برای هر دو زبان تهیه می‌کنیم. 

In [118]:
from gensim.models import KeyedVectors

# Load the mapped embeddings
en_embeddings = KeyedVectors.load_word2vec_format('./model/en_mapped_embeddings.txt', binary=False)
fa_embeddings = KeyedVectors.load_word2vec_format('./model/fa_mapped_embeddings.txt', binary=False)

persian_df = pd.read_csv('./data/persian_df.csv')
english_df = pd.read_csv('./data/english_df.csv')


In [119]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
import re

# Initialize the spell checker
spell = SpellChecker()

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a set of stopwords to remove
stop_words = set(stopwords.words('english'))

def preprocess_query(query, language, embeddings):
    # Lowercase the query
    query = query.lower()
    
    tokens = word_tokenize(query)
    
    # Correct misspellings in the tokens
    tokens = [spell.correction(token) if spell.correction(token) is not None else token for token in tokens]
    
    # Remove stopwords from the tokens
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Initialize an empty list to store the token vectors
    token_vectors = []
    
    # For each token in the query
    for token in tokens:
        # Look up the vector representation of the token in the embeddings
        try:
            token_vector = embeddings.get_vector(token)
            token_vectors.append(token_vector)
        except KeyError:
            # If the token is not in the embeddings, skip it
            pass
    
    # Compute the average of the token vectors to get a single vector representation of the query
    if len(token_vectors) > 0:
        query_vector = np.mean(token_vectors, axis=0)
    else:
        # If none of the tokens were found in the embeddings, return a zero vector
        query_vector = np.zeros(embeddings.vector_size)
    
    normalized_query = ' '.join(tokens)
    return normalized_query, query_vector


In [120]:
def preprocess_row(row, language, embeddings, preprocessed_rows, vectorized_rows):
    # Lowercase the query
    row = row.str.lower()
    
    row = row.fillna(' ').astype(str)
    tokens = word_tokenize(' '.join(row))
    
    # Remove stopwords from the tokens
    tokens = [token for token in tokens if token not in stop_words]
    
    
    # Initialize an empty list to store the token vectors
    token_vectors = []
    
    # For each token in the query
    for token in tokens:
        # Look up the vector representation of the token in the embeddings
        try:
            token_vector = embeddings.get_vector(token)
            token_vectors.append(token_vector)
        except KeyError:
            # If the token is not in the embeddings, skip it
            pass
    
    # Compute the average of the token vectors to get a single vector representation of the query
    if len(token_vectors) > 0:
        row_vector = np.mean(token_vectors, axis=0)
    else:
        # If none of the tokens were found in the embeddings, return a zero vector
        row_vector = np.zeros(embeddings.vector_size)
    
    preprocessed_rows.append(row)
    vectorized_rows.append(row_vector)


In [121]:
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import cosine_similarity

def calc_rw(dataset, language, embeddings):
    
    preprocessed_rows = []
    vectorized_rows = []
    
    dataset.apply(lambda r: preprocess_row(r, language, embeddings, preprocessed_rows, vectorized_rows), axis=1)
        
    vectorized_rows = np.array(vectorized_rows)
    return preprocessed_rows, vectorized_rows

preprocessed_rows, vectorized_rows = calc_rw(english_df, 'en', en_embeddings)


In [122]:

def calc_fuzzy(query, preprocessed_rows):
    fuzzy_scores = []
    for row in preprocessed_rows:
        row = row.str.lower()
        fuzzy_score = fuzz.token_set_ratio(query, ' '.join(row))
        fuzzy_scores.append(fuzzy_score)
    return fuzzy_scores

def search(query, language, dataset, embeddings, top_k=10):
    query, query_vector = preprocess_query(query, language, embeddings)
    
    similarities = cosine_similarity(query_vector.reshape(1, -1), vectorized_rows)
    fuzzy_scores = calc_fuzzy(query, preprocessed_rows)
    
    # Combine the cosine similarity scores and the fuzzy match scores into a single score
    scores = 0.75 * similarities[0] + 0.25 * np.array(fuzzy_scores)
    
    # Get the indices of the top_k highest-scoring rows
    top_k_indices = np.argsort(scores)[::-1][:top_k]
    
    # Return the top_k highest-scoring rows from the original dataset
    return dataset.iloc[top_k_indices]

# Example usage
query = 'aspirin'
language = 'en'
top_k_results = search(query, language, english_df, en_embeddings, top_k=10)
top_k_results


,Unnamed: 0.1,Unnamed: 0,pharma_company,brand_name_en,generic_name_en,substance_name_en,atc_code_raw,anatomical_desc,therapeutic_desc,pharmacological_desc,subpharmacological_desc,chemical_desc,preprocessed_index
14346,14346,14346,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,14346
14348,14348,14348,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,14348
55118,55118,55118,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,55118
55117,55117,55117,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,55117
55116,55116,55116,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,55116
14347,14347,14347,pars darou,aspirin tablet oral 325 mg,asa (acetylsalicylic acid) tablet oral 325 mg,asa (acetylsalicylic acid),n02ba01,nervous system,analgesics,other analgesics and antipyretics,salicylic acid and derivatives,acetylsalicylic acid,14347
47949,47949,47949,pars darou,"aspirin tablet, delayed release oral 81 mg","asa (acetylsalicylic acid) tablet, delayed release oral 81 mg",asa (acetylsalicylic acid),b01ac06,blood and blood forming organs,antithrombotic agents,antithrombotic agents,platelet aggregation inhibitors excl. heparin,acetylsalicylic acid,47949
30304,30304,30304,pars darou,"aspirin tablet, delayed release oral 81 mg","asa (acetylsalicylic acid) tablet, delayed release oral 81 mg",asa (acetylsalicylic acid),b01ac06,blood and blood forming organs,antithrombotic agents,antithrombotic agents,platelet aggregation inhibitors excl. heparin,acetylsalicylic acid,30304
34693,34693,34693,pars darou,"aspirin tablet, delayed release oral 81 mg","asa (acetylsalicylic acid) tablet, delayed release oral 81 mg",asa (acetylsalicylic acid),b01ac06,blood and blood forming organs,antithrombotic agents,antithrombotic agents,platelet aggregation inhibitors excl. heparin,acetylsalicylic acid,34693
44468,44468,44468,"no.115 , jalal st ., esteqlal town ,tehran","aspirin tablet, delayed release oral 81 mg","asa (acetylsalicylic acid) tablet, delayed release oral 81 mg",asa (acetylsalicylic acid),b01ac06,blood and blood forming organs,antithrombotic agents,antithrombotic agents,platelet aggregation inhibitors excl. heparin,acetylsalicylic acid,44468


In [123]:
def run(query, k=10):
    top_k_results = search(query, 'en', english_df, en_embeddings, top_k=k)
    return top_k_results

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## **ارزیابی عملکرد سامانه**
برای ارزیابی عملکرد سامانه، برای هر یک از زبان‌های انگلیسی و فارسی،‌ چند عبارت که به اشکال مختلف یک دارو را توصیف می‌کنند، جستجو شده‌اند. این عبارات، شامل عباراتی با املای صحیح و غلط، نوشتن شکل جنریک دارو یا نام تجاری آن (مثلا ایبوبروفن یا ژلوفن)، نوشتن دسته‌بندی دارو (مثلا داروی ضد افسردگی)، و کاربرد دارو می‌باشند. برای هر زبان، چندین کوئری از این دست عبارات طراحی شده که نتایج این کوئری‌ها، برای این عبارات در ادامه نمایش داده می‌شود.  

In [124]:
def display_input_results(test_inputs):
    for input in test_inputs:
        print('Query:', input)
        results = run(input)
        print(results)
        print('\n')

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### **زبان انگلیسی**

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام جنریک دارو به صورت صحیح 

In [125]:

test_inputs = ['Sertraline', 'sertraline', 'Propranolol', 'Gabapentin']
display_input_results(test_inputs)

Query: Sertraline
       Unnamed: 0.1  Unnamed: 0     pharma_company                   brand_name_en                                  generic_name_en              substance_name_en atc_code_raw anatomical_desc  therapeutic_desc pharmacological_desc                  subpharmacological_desc chemical_desc  preprocessed_index
53060         53060       53060  jaber ebne hayyan  sertraline   tablet oral 50 mg  sertraline (as hydrochloride) tablet oral 50 mg  sertraline (as hydrochloride)      n06ab06  nervous system  psychoanaleptics      antidepressants  selective serotonin reuptake inhibitors    sertraline               53060
37056         37056       37056  jaber ebne hayyan  sertraline   tablet oral 50 mg  sertraline (as hydrochloride) tablet oral 50 mg  sertraline (as hydrochloride)      n06ab06  nervous system  psychoanaleptics      antidepressants  selective serotonin reuptake inhibitors    sertraline               37056
8653           8653        8653  jaber ebne hayyan  sertraline  

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام تجاری دارو به صورت صحیح 

In [126]:

test_inputs = ['gabakim', 'asentra', 'Asentra']
display_input_results(test_inputs)

Query: gabakim
       Unnamed: 0.1  Unnamed: 0            pharma_company                  brand_name_en                 generic_name_en substance_name_en atc_code_raw anatomical_desc therapeutic_desc pharmacological_desc subpharmacological_desc chemical_desc  preprocessed_index
1265           1265        1265  hakim pharmaceutical co.  gabakim   capsule oral 300 mg  gabapentin capsule oral 300 mg        gabapentin      n03ax12  nervous system   antiepileptics       antiepileptics    other antiepileptics    gabapentin                1265
13772         13772       13772  hakim pharmaceutical co.  gabakim   capsule oral 300 mg  gabapentin capsule oral 300 mg        gabapentin      n03ax12  nervous system   antiepileptics       antiepileptics    other antiepileptics    gabapentin               13772
9427           9427        9427  hakim pharmaceutical co.  gabakim   capsule oral 300 mg  gabapentin capsule oral 300 mg        gabapentin      n03ax12  nervous system   antiepileptics       an

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام جنریک دارو با غلط‌های املایی 

In [127]:
test_inputs = ['propranol', 'propranololol', 'ibubrophen']
display_input_results(test_inputs)

Query: propranol
       Unnamed: 0.1  Unnamed: 0         pharma_company                           brand_name_en                              generic_name_en          substance_name_en atc_code_raw anatomical_desc therapeutic_desc pharmacological_desc subpharmacological_desc chemical_desc  preprocessed_index
54970         54970       54970             pars darou         flurazepam   capsule oral 15 mg                flurazepam capsule oral 15 mg                 flurazepam          NaN  nervous system              NaN                  NaN                     NaN           NaN               54970
50455         50455       50455             pars darou         flurazepam   capsule oral 15 mg                flurazepam capsule oral 15 mg                 flurazepam          NaN  nervous system              NaN                  NaN                     NaN           NaN               50455
5916           5916        5916            pfizer inc.             rapamune   tablet oral 1 mg             

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### دسته‌بندی کلی دارو

In [128]:
test_inputs = ['anti inflamatory', 'pain killer', 'antidepressant']
display_input_results(test_inputs)

Query: anti inflamatory
       Unnamed: 0.1  Unnamed: 0                  pharma_company                                                 brand_name_en                                            generic_name_en substance_name_en atc_code_raw                  anatomical_desc             therapeutic_desc      pharmacological_desc                                             subpharmacological_desc            chemical_desc  preprocessed_index
47748         47748       47748  arnikahealth pharmaceutical co  anti hemorrhoid   suppository rectal 60 mg/5 mg/50 mg/400 mg  antihemorrhoid suppository rectal 60 mg/5 mg/50 mg/400 mg    antihemorrhoid      n01bb52                   nervous system                  anesthetics        anesthetics, local                                                              amides  lidocaine, combinations               47748
16902         16902       16902  arnikahealth pharmaceutical co  anti hemorrhoid   suppository rectal 60 mg/5 mg/50 mg/400 mg  antihemorrhoid 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### دسته‌بندی جزئی‌تر دارو

In [129]:
test_inputs = ['selective serotonin reuptake inhibitor', 'tri cyclic antidepressant', 'non steroidal anti inflammatory drug']
display_input_results(test_inputs)

Query: selective serotonin reuptake inhibitor
       Unnamed: 0.1  Unnamed: 0                                          pharma_company                         brand_name_en                                  generic_name_en              substance_name_en atc_code_raw anatomical_desc  therapeutic_desc pharmacological_desc                  subpharmacological_desc chemical_desc  preprocessed_index
5761           5761        5761  sandoz syntek ilaç hammaddeleri sanayi ve ticaret a.s.   sertralin hexal   tablet oral 50 mg  sertraline (as hydrochloride) tablet oral 50 mg  sertraline (as hydrochloride)      n06ab06  nervous system  psychoanaleptics      antidepressants  selective serotonin reuptake inhibitors    sertraline                5761
63799         63799       63799                 shafa pharmaceutical & hygienic mfg.co.  sertraline-shafa   tablet oral 50 mg  sertraline (as hydrochloride) tablet oral 50 mg  sertraline (as hydrochloride)      n06ab06  nervous system  psychoanaleptics    

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

### **زبان فارسی**

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام جنریک دارو به صورت صحیح 

In [130]:

test_inputs = ['سرترالین', 'پروپرانولول', 'گاباپنتین']
display_input_results(test_inputs)

Query: سرترالین
       Unnamed: 0.1  Unnamed: 0         pharma_company                                                    brand_name_en                                                                                  generic_name_en                                        substance_name_en atc_code_raw     anatomical_desc             therapeutic_desc        pharmacological_desc     subpharmacological_desc chemical_desc  preprocessed_index
42448         42448       42448    نوشا فارمد ایرانیان                                 lipzib   tablet oral 40 mg/10 mg                                    atorvastatin (as calcium) / ezetimibe tablet oral 40 mg/10 mg                    atorvastatin (as calcium) / ezetimibe          NaN      nervous system                          NaN                         NaN                         NaN           NaN               42448
34033         34033       34033  فن آوری زیستی رزفارمد                    dalfyra   tablet, extended release oral 10 mg             

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام تجاری دارو به صورت صحیح 

In [131]:

test_inputs = ['آسنترا', 'گاباکیم', 'ژلوفن']
display_input_results(test_inputs)

Query: آسنترا
       Unnamed: 0.1  Unnamed: 0       pharma_company                                         brand_name_en                                                                                  generic_name_en                                        substance_name_en atc_code_raw  anatomical_desc            therapeutic_desc        pharmacological_desc subpharmacological_desc chemical_desc  preprocessed_index
33961         33961       33961    نیواد فارمد سلامت          fluguard   injection parenteral 1 {dose}  ml                        influenza vaccine, inactivated, tetravalent injection parenteral 1 {dose}              influenza vaccine, inactivated, tetravalent          NaN   nervous system                         NaN                         NaN                     NaN           NaN               33961
42448         42448       42448  نوشا فارمد ایرانیان                      lipzib   tablet oral 40 mg/10 mg                                    atorvastatin (as calcium) / ezetim

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### نام جنریک دارو با غلط‌های املایی 

In [132]:

test_inputs = ['سرتارلین', 'پروپانول', 'ملاتینون']
display_input_results(test_inputs)

Query: سرتارلین
       Unnamed: 0.1  Unnamed: 0         pharma_company                                                    brand_name_en                                                                                  generic_name_en                                        substance_name_en atc_code_raw     anatomical_desc             therapeutic_desc        pharmacological_desc     subpharmacological_desc chemical_desc  preprocessed_index
42448         42448       42448    نوشا فارمد ایرانیان                                 lipzib   tablet oral 40 mg/10 mg                                    atorvastatin (as calcium) / ezetimibe tablet oral 40 mg/10 mg                    atorvastatin (as calcium) / ezetimibe          NaN      nervous system                          NaN                         NaN                         NaN           NaN               42448
34033         34033       34033  فن آوری زیستی رزفارمد                    dalfyra   tablet, extended release oral 10 mg             

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### دسته‌بندی کلی دارو

In [133]:

test_inputs = ['ضد افسردگی', 'مسکن', 'ملین']
display_input_results(test_inputs)

Query: ضد افسردگی
       Unnamed: 0.1  Unnamed: 0         pharma_company                                         brand_name_en                                                                                  generic_name_en                                        substance_name_en atc_code_raw anatomical_desc            therapeutic_desc        pharmacological_desc     subpharmacological_desc chemical_desc  preprocessed_index
42447         42447       42447    نوشا فارمد ایرانیان                                  lipzib   tablet oral                                                atorvastatin (as calcium) / ezetimibe tablet oral                    atorvastatin (as calcium) / ezetimibe          NaN  nervous system                         NaN                         NaN                         NaN           NaN               42447
42448         42448       42448    نوشا فارمد ایرانیان                      lipzib   tablet oral 40 mg/10 mg                                    atorvastatin (as c

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### دسته‌بندی جزئی‌تر دارو

In [134]:

test_inputs = ['مهارکننده انتخابی بازجذب سروتونین', 'ضد التهاب غیر استروئیدی']
display_input_results(test_inputs)

Query: مهارکننده انتخابی بازجذب سروتونین
       Unnamed: 0.1  Unnamed: 0         pharma_company                                         brand_name_en                                                                                  generic_name_en                                        substance_name_en atc_code_raw  anatomical_desc            therapeutic_desc        pharmacological_desc     subpharmacological_desc chemical_desc  preprocessed_index
42447         42447       42447    نوشا فارمد ایرانیان                                  lipzib   tablet oral                                                atorvastatin (as calcium) / ezetimibe tablet oral                    atorvastatin (as calcium) / ezetimibe          NaN   nervous system                         NaN                         NaN                         NaN           NaN               42447
42448         42448       42448    نوشا فارمد ایرانیان                      lipzib   tablet oral 40 mg/10 mg                             